# PolymerXtal - Polymer Crystal Structure Generator and Analysis Software

In [ ]:
from hublib import use
from hublib.cmd import runCommand

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

polymer_type:
    type: Text
    value: PAN
    description: Select the polymer type.
    options: [PAN, PE, PP, PS, POM, PTFE, PVC]

helice_atoms:
    type: Integer
    description: Number of chain (backbone) atoms of a motif of the helix
    value: 2
    min: 2
    max: 2

helice_motifs:
    type: Integer
    description: Number of motifs of the helix
    value: 3
    min: 1
    max: 3

helice_turns:
    type: Integer
    description: Number of motifs of the helix
    value: 1
    min: 1
    max: 1

monomers:
    type: Number
    description: Number of monomers
    value: 40
    min: 1
    max: 5000

tacticity:
    type: Text
    value: isotactic
    description: Select the tacticity of the polymer chain.
    options: [isotactic, atactic, syndiotactic, None]

chiriality:
    type: Text
    value: right
    description: Select the chiriality of the polymer chain.
    options: [left, right, None]

head_tail_defect_ratio:
    type: Number
    description: Ratio of head-to-head and tail-to-tail connections
    value: 0
    min: 0
    max: 1
        
configs:
    type: Number
    description: Number of attempts to find a configuration
    value: 30
    min: 1
    max: 5000


In [ ]:
polymer_type = "PAN"
helice_atoms = 2
helice_motifs = 3
helice_turns = 1
monomers = 40
tacticity = "isotactic"
chiriality = "right"
head_tail_defect_ratio = 0
configs = 30

In [ ]:
EXTRA_FILES = ['polymerxtal']

In [ ]:
import os
import sys
from polymerxtal import Helice, Chain
from simtool import DB

In [ ]:
chain=Chain(polymer_type=polymer_type,helice=Helice(helice_atoms,helice_motifs,helice_turns),num_monomers=monomers,tacticity=tacticity,
            chiriality=chiriality,head_tail_defect_ratio=head_tail_defect_ratio)
helix_name=chain.build_chain()

In [ ]:
%%yaml OUTPUTS
Helix Name:
    type: Text
    
Output PDBfile:
    type: Text   

Output PDB:
    type: Text

In [ ]:
import os

def copyAndSaveFileAsOutput(outputVariableName,inputPath):
    if inputPath.startswith('file://'):
        resultFile = os.path.basename(inputPath[7:])
        if resultFile != inputPath[7:]:
            shutil.copy2(inputPath[7:],resultFile)
    else:
        resultFile = os.path.basename(inputPath)
        if resultFile != inputPath:
            shutil.copy2(inputPath,resultFile)
    db.save(outputVariableName,file=resultFile)

In [ ]:
db = DB(OUTPUTS)
print(db)
db.save('Helix Name', helix_name)
copyAndSaveFileAsOutput('Output PDBfile', helix_name+'.pdb')
output_pdb = ""
with open(helix_name+'.pdb', 'r') as input_text:
    output_pdb = input_text.read()
db.save('Output PDB', output_pdb)